In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [ ]:
def rotate_image(image, angle):
    h, w = image.shape[:2]
    M = cv2.getRotationMatrix2D((w/2,h/2), angle, 1.0)
    return cv2.warpAffine(image, M, (w, h), borderMode=cv2.BORDER_REFLECT)

def change_brightness(image, factor):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV).astype(np.float32)
    hsv[...,2] = np.clip(hsv[...,2] * factor, 0, 255)
    return cv2.cvtColor(hsv.astype(np.uint8), cv2.COLOR_HSV2BGR)

def add_gaussian_noise(image, mean=0, sigma=15):
    noise = np.random.normal(mean, sigma, image.shape).astype(np.float32)
    noisy = image.astype(np.float32) + noise
    return np.clip(noisy, 0, 255).astype(np.uint8)

In [ ]:
def augment_and_save(input_dir, output_dir,
                     n_rotations=2, n_brightness=2, n_noise=2):
    """
      - n_rotations випадкових обертів
      - n_brightness випадкових зміни яскравості
      - n_noise випадкових варіантів з шумом
    """
    os.makedirs(output_dir, exist_ok=True)
    for cls in os.listdir(input_dir):
        src_cls = os.path.join(input_dir, cls)
        dst_cls = os.path.join(output_dir, cls)
        if not os.path.isdir(src_cls):
            continue
        os.makedirs(dst_cls, exist_ok=True)

        for fname in tqdm(os.listdir(src_cls), desc=f"Class {cls}"):
            if not fname.lower().endswith(('.jpg','.png','.bmp','jpeg')):
                continue
            img_path = os.path.join(src_cls, fname)
            img = cv2.imread(img_path)
            if img is None:
                continue

            # 1) копіюємо оригінал
            base, ext = os.path.splitext(fname)
            cv2.imwrite(os.path.join(dst_cls, f"{base}_orig{ext}"), img)

            # 2) обертання
            for i in range(n_rotations):
                angle = np.random.uniform(-30, 30)
                aug = rotate_image(img, angle)
                cv2.imwrite(os.path.join(dst_cls, f"{base}_rot{i}{ext}"), aug)

            # 3) яскравість
            for i in range(n_brightness):
                factor = np.random.uniform(0.7, 1.3)
                aug = change_brightness(img, factor)
                cv2.imwrite(os.path.join(dst_cls, f"{base}_bright{i}{ext}"), aug)

            # 4) шум
            for i in range(n_noise):
                aug = add_gaussian_noise(img, mean=0, sigma=15)
                cv2.imwrite(os.path.join(dst_cls, f"{base}_noise{i}{ext}"), aug)

In [ ]:
augment_and_save(
    input_dir="/content/drive/MyDrive/105_classes_pins_dataset",
    output_dir="/content/drive/MyDrive/105_classes_pins_dataset_augmented",
    n_rotations=3,
    n_brightness=3,
    n_noise=3
)

Class models: 100%|██████████| 6/6 [00:00<00:00, 40524.68it/s]
